In [1]:
# Import necessary libraries
import pandas as pd
import json
import string 

# Load the CSV dataset
df = pd.read_csv("semi_strut.csv")

# Display the first few rows of the dataset
df.head()

,Document ID,Content
0,1,"{\r\n ""title"": ""Introduction to Python"",\r\n..."
1,2,"{\r\n ""title"": ""Data Analysis with Pandas"",\..."
2,3,"{\r\n ""title"": ""Web Development with Flask"",..."
3,4,"{\r\n ""title"": ""Machine Learning with Scikit..."
4,5,"{\r\n ""title"": ""Data Visualization with Matp..."


In [3]:
# Tokenization function to extract terms from the JSON-like content
# Remember to exact both 
# 1 .Extract terms from various fields (title, author)
def tokenize_content(content):
    content_dict = json.loads(content)
    terms = []
    
    # Extract terms from various fields (title, author)
    terms.extend(content_dict.get("title", "").split())
    # terms.extend(content_dict.get("author", "").split())
    return terms
tokenize_content(df["Content"][0])
# 2. apply to all row in panda df , by create new column "Terms"
df["Terms"] = df["Terms"] = df["Content"].apply(tokenize_content)
df

,Document ID,Content,Terms
0,1,"{\r\n ""title"": ""Introduction to Python"",\r\n...","[Introduction, to, Python]"
1,2,"{\r\n ""title"": ""Data Analysis with Pandas"",\...","[Data, Analysis, with, Pandas]"
2,3,"{\r\n ""title"": ""Web Development with Flask"",...","[Web, Development, with, Flask]"
3,4,"{\r\n ""title"": ""Machine Learning with Scikit...","[Machine, Learning, with, Scikit-Learn]"
4,5,"{\r\n ""title"": ""Data Visualization with Matp...","[Data, Visualization, with, Matplotlib]"


In [5]:
# 4. Implement a preprocessing function that converts terms to lowercase, removes punctuation, and removes common stop words.
    # Create another new column "Terms_preprocessed"
def preprocess_terms(terms):
    # Define a set of common stop words
    stop_words = set([
        "a", "an", "the", "and", "is", "in", "it", "to", "of", "for", "on", "with", "as"
    ])
    
    # Remove punctuation and convert to lowercase
    terms = [term.lower().strip(string.punctuation) for term in terms]
    
    # Remove stop words
    terms = [term for term in terms if term not in stop_words]
    
    return terms

# your code here....
df["Terms_preprocessed"] = df["Terms"].apply(preprocess_terms)
df

,Document ID,Content,Terms,Terms_preprocessed
0,1,"{\r\n ""title"": ""Introduction to Python"",\r\n...","[Introduction, to, Python]","[introduction, python]"
1,2,"{\r\n ""title"": ""Data Analysis with Pandas"",\...","[Data, Analysis, with, Pandas]","[data, analysis, pandas]"
2,3,"{\r\n ""title"": ""Web Development with Flask"",...","[Web, Development, with, Flask]","[web, development, flask]"
3,4,"{\r\n ""title"": ""Machine Learning with Scikit...","[Machine, Learning, with, Scikit-Learn]","[machine, learning, scikit-learn]"
4,5,"{\r\n ""title"": ""Data Visualization with Matp...","[Data, Visualization, with, Matplotlib]","[data, visualization, matplotlib]"


In [8]:
# Initialize an empty inverted index dictionary
# Build the inverted index
# Initialize an empty inverted index dictionary
inverted_index = {}

# Build the inverted index
for index, row in df.iterrows():
    document_id = row["Document ID"]
    terms = row["Terms_preprocessed"]
    
    # Update the inverted index with terms and document IDs
    for term in terms:
        if term not in inverted_index:
            inverted_index[term] = set()
        inverted_index[term].add(document_id)

# Display the inverted index
inverted_index

{'introduction': {1},
 'python': {1},
 'data': {2, 5},
 'analysis': {2},
 'pandas': {2},
 'web': {3},
 'development': {3},
 'flask': {3},
 'machine': {4},
 'learning': {4},
 'scikit-learn': {4},
 'visualization': {5},
 'matplotlib': {5}}

In [10]:
#Or 'phyton', 'pandas'
def or_postings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    while p1<len(posting1) and p2<len(posting2):
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1+=1
            p2+=1
        elif posting1[p1]>posting2[p2]:
            result.append(posting2[p2])
            p2+=1
        else:
            result.append(posting1[p1])
            p1+=1
    while p1 < len(posting1):
        result.append(posting1[p1])
        p1+=1
    while p2 < len(posting2):
        result.append(posting2[p2])
        p2+=1
    return result

pl_1 = list(inverted_index['python'])
pl_2 = list(inverted_index['data'])
or_postings(pl_1,pl_2)

[1, 2, 5]

In [14]:
# 2. "Python" AND "data"
def and_postings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    while p1 < len(posting1) and p2 <len(posting2):
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1+=1
            p2+=1
        elif posting1[p1] > posting2[p2]:
            p2+=1
        else:
            p1+=1
    return result
pl_1 = list(inverted_index['python'])
pl_2 = list(inverted_index['data'])
and_postings(pl_1, pl_2)

[]